In [114]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

In [115]:
df = pd.read_csv('../data/03_primary/cleanData.csv')

/tmp/ipykernel_954473/3882104848.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/03_primary/cleanData.csv')


In [116]:
df.head()

,invoice_line_no,date,store,name,address,city,zipcode,county,category,category_name,...,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons,longitude,latitude
0,INV-48770300006,2022-06-29T00:00:00.000,5900,NEW STAR FLETCHER / WATERLOO,315 FLETCHER AVE,WATERLOO,50701.0,BLACK HAWK,1031100.0,AMERICAN VODKAS,...,24,375,3.50,5.25,24,126.00,9.0,2.37,-92.367142,42.489119
1,INV-48782900001,2022-06-29T00:00:00.000,3390,OKOBOJI AVENUE LIQUOR,1610 OKOBOJI AVENUE,MILFORD,51351.0,DICKINSON,1031100.0,AMERICAN VODKAS,...,12,1000,3.97,5.96,288,1716.48,288.0,76.08,-95.149955,43.331525
2,INV-48703200006,2022-06-27T00:00:00.000,4604,PIT STOP LIQUORS / NEWTON,"1324, 1ST AVE E",NEWTON,50208.0,JASPER,1062200.0,WHITE RUM,...,24,375,5.00,7.50,24,162.00,9.0,2.37,-93.035653,41.699173
3,INV-48692600064,2022-06-27T00:00:00.000,2552,HY-VEE FOOD STORE #3 / CEDAR RAPIDS,20 WILSON AVENUE WEST,CEDAR RAPIDS,52404.0,LINN,1011100.0,BLENDED WHISKIES,...,6,1750,13.99,20.99,6,116.64,10.5,2.77,-91.658576,41.955859
4,INV-48679200008,2022-06-24T00:00:00.000,2523,HY-VEE FOOD STORE / CENTERVILLE,609 N 18TH,CENTERVILLE,52544.0,APPANOOSE,1081300.0,AMERICAN CORDIALS & LIQUEURS,...,12,750,8.66,12.99,12,155.88,9.0,2.37,-92.867967,40.737577


In [117]:
label = 'state_bottle_retail'

In [118]:
train_data, test_data = train_test_split(df, test_size=0.25,random_state=42)

In [119]:
predictor = TabularPredictor(label=label, path='../asi/src/asi/models/09_gluon')

In [120]:
hyperparameters = {'XGBoost': {}, 'LightGBM': {}}
time_limit = 300  # 5 minut na model

In [121]:
predictor.fit(train_data, presets='medium_quality', time_limit=time_limit)
# No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
# 	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
# 	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
# 	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
# 	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
# 	presets='medium_quality' : Fast training time, ideal for initial prototyping.

Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "../asi/src/asi/models/09_gluon"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #38~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Nov  2 18:01:13 UTC 2
CPU Count:          16
Memory Avail:       4.71 GB / 30.71 GB (15.3%)
Disk Space Avail:   267.83 GB / 466.23 GB (57.4%)
Train Data Rows:    726620
Train Data Columns: 23
Label Column:       state_bottle_retail
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (9702.0, 0.0, 16.32443, 22.9032)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclas

[1000]	valid_set's rmse: 4.53692
[2000]	valid_set's rmse: 1.92737
[3000]	valid_set's rmse: 1.64544


	Ran out of time, early stopping on iteration 3989. Best iteration is:
	[3647]	valid_set's rmse: 1.60781
	-1.6078	 = Validation score   (-root_mean_squared_error)
	157.73s	 = Training   runtime
	1.06s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 158.87s of the -2.76s of remaining time.
	Ensemble Weights: {'LightGBMXT': 1.0}
	-1.6078	 = Validation score   (-root_mean_squared_error)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 304.62s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("../asi/src/asi/models/09_gluon")


In [122]:
performance = predictor.evaluate(test_data)

In [123]:
y_pred = predictor.predict(test_data)

In [124]:
print(performance)

{'root_mean_squared_error': -9.538560832272871, 'mean_squared_error': -90.98414275097015, 'mean_absolute_error': -0.34254594159089746, 'r2': 0.8197761840652396, 'pearsonr': 0.9222124426824944, 'median_absolute_error': -0.08489107131958029}
